In [8]:
from tree_lstm import TreeLSTM, Tree, BatchedTree
import json
from sentence_transformers import SentenceTransformer
from lxml import html, etree

In [43]:
class Node:
    def __init__(self, parent_id = None):
        self.children = []
        self.data = ""
        self.embedding = None
        self.id = None
        self.parent_id = parent_id
    def add_child(self, child):
        self.children.append(child)
    def set_data(self, tag, text):
        self.data = ""
        if tag is etree.Comment:
            self.data += "comment"
        elif tag != None:
            self.data += tag
                
        
        # print(tag)
        # print(text)     
        if text != None:
            self.data += text

    def set_id(self, id):
        self.id = id
    

model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')
H_SIZE = 30
X_SIZE = 128


def loadData():
    f = open('../Dataset/data.json', 'r')
    val = f.read()
    f.close()
    d = json.loads(val)
    return d

def idTree(htmlTree):
    nodeIds = dict()
    count = 0
    for i in htmlTree.getiterator():
        nodeIds[i] = count
        count += 1
    return nodeIds

def convertCodeToTree(code):
    t = html.fromstring(code)
    tree = Node()

    nodeIds = idTree(t)
    
    # Want dict mapping object -> id


    def parseTree(htmlTree, treeObject):
        tree_id = nodeIds[htmlTree]
        treeObject.set_data(htmlTree.tag, htmlTree.text)
        treeObject.set_id(tree_id)
        for treeChild in htmlTree.getchildren():
            new_tree = Node(parent_id = tree_id)
            parseTree(treeChild, new_tree)
            treeObject.add_child(new_tree)
        return treeObject

    return parseTree(t, tree)


def embbedNode(nodes):
    nodes.embeddings =  model.encode([nodes.data])[0]
    for i in nodes.children:
        embbedNode(i)

def makeTree(t):
    finTree = Tree(H_SIZE)
    

    def addNodes(root):
        finTree.add_node(parent_id = root.parent_id, tensor=root.embedding)
        for i in root.children:
            addNodes(i)

    addNodes(t)
    return finTree

def batchTrees(treeList):
    return BatchedTree(treeList)

def trainLSTM(data):
    model = TreeLSTM(x_size=X_SIZE, h_size=H_SIZE, dropout=0.3, cell_type='child_sum')
    return model(data)


, te

SyntaxError: '(' was never closed (2417899904.py, line 16)

In [4]:
data = loadData()
treeList = []
for i in data['urls']:
    trep = convertCodeToTree(data['code'][i])
    embbedNode(trep)
    lstmTree = makeTree(trep)
    treeList.append(lstmTree)

lstmTreeBatch = batchTrees(treeList)



In [9]:
model = trainLSTM(lstmTreeBatch)

In [19]:
data = loadData()

In [41]:
test = data['code'][data['urls'][0]]
# t = html.fromstring(test)
convertCodeToTree(test)

<cyfunction Comment at 0x29c9c97d0>
<cyfunction Comment at 0x29c9c97d0>
<cyfunction Comment at 0x29c9c97d0>
<cyfunction Comment at 0x29c9c97d0>


In [42]:
for i in t.getchildren():
    print(i.tag)

head
body
